# TODO:
#### Automatic data retrieval from google forms for during-suvrvey checking
[X] spreadsheets url inference <br>
[X] gather results in python script

#### Qualification filtering
[ ] ~50 qualifications

#### Frauder detection
[X] > X ratio repeated ===> discarded

#### Honeypots
[X] place honeypots from a defined set in each form<br>
[X] honeypot frauders detection

#### MTurk templating
[X] Create guidelines for the workers

#### Gform templating
[X] Remove index of the emojis and add separator btwn each question NB: not practical

#### Emojis png
[X] Rerun creation of emojis png + upload on drive


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import create_mturk_client,monitor_worker_tags,clean_own_worker,Turker, Watcher
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results
from src.data.fraudulous import detect_repeat_frauders,detect_honey_frauders
from src.utils import generate_password 
from src.constants import NMB_FORMS_THRESHOLD,TOKEN_PATH,CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH, FORMS_RESULTS_DIR,HONEYPOTS
from src.utils import read_access_keys
import pandas as pd
import jdc
import pickle as pk
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

## Urls info retrieval

In [3]:
############ TODO: copy-paste the id of the most recent version of the url file ############
file_id = "1JFLNnl1Mw5J2aKFql66D17NgqqN59xRz6tOij-Ld7pE"
############################################################################################
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
download_drive_txt(URL_INDEX_PATH,file_id,service)  # download the most recent url_index
url_index = pd.read_csv(URL_INDEX_PATH,sep=",",header=None,names=['url','res_url'])
url_forms = url_index.url.to_dict()
url_results = url_index.res_url.to_dict()

Download 100%


In [4]:
download_all_csv_results(url_results,FORMS_RESULTS_DIR,service)

Download 100%
Download 100%
Download 100%
Download 100%


## MTurk management

In [5]:
turk = Turker(hitlayout="3VQCRCGMCT2NU7RBSKK6J2PIFKVQ77",
             MaxAssignments = 10,
             LifetimeInSeconds = 600,
             AutoApprovalDelayInSeconds=600,
             AssignmentDurationInSeconds=600,
             Reward='0.01',)

Loading hit2form


In [9]:
turk.create_forms_hits(url_forms)

Creating hit for form 0
Creating hit for form 1
Creating hit for form 2
Creating hit for form 3


In [13]:
turk.list_hits()

Expiration:Nov 26 2020 19:33:12 (9.63 minutes left)


,HITId,Status,Completed,Percent_completed
FormIdx,,,,
0,36818Z1KV3YUPD71WIZPXY0ZW193AV,Assignable,0,0
1,3ZZAYRN1I6COQYZWLGAYG6O2FHPTO0,Assignable,0,0
2,3YGE63DIN8IVFYB7E0DYDPZJL8RW03,Assignable,0,0
3,35O6H0UNLS184LX9470V5U120KU5J3,Assignable,0,0


In [10]:
clean_own_worker(turk.client)

Worker already clean


In [12]:
turk.get_url("36818Z1KV3YUPD71WIZPXY0ZW193AV")

'https://workersandbox.mturk.com/mturk/preview?groupId=3BW6RX62GFQF0UCE0YFLZL7NB2LLEY'

In [24]:
watcher = Watcher(form_results_path=FORMS_RESULTS_DIR,
                  url_results=url_results,
                  drive_service=service,
                 threshold=2)

In [25]:
watcher.monitor()

0 th iteration
Download 100%
Download 100%
Download 100%
Download 100%
Tagging worker WORKER0


ClientError: An error occurred (ValidationException) when calling the AssociateQualificationWithWorker operation: 1 validation error detected: Value 'WORKER0' at 'workerId' failed to satisfy constraint: Member must satisfy regular expression pattern: ^A[A-Z0-9]+$

In [9]:
turk.list_all_results()

No results ready yet for 31SIZS5W590EVVZKW4X75WIK6XAQRS
No results ready yet for 31KPKEKW4AY3RVDWJ9H2QBMYB7A0BS
No results ready yet for 3ATYLI1PRTNUCV3R9MWUE01THFYOJQ
No results ready yet for 3T2EL38U0M5YSRLW533ZUE487EVXQV
No results ready yet for 3THR0FZ95PE07LRRC787UFELL42LOO
No results ready yet for 3XEIP58NL08C6P5YEGJOADLP585ZL4


,WorkerId,HITId,Answer,Code,Status
0,A29C1XYH77RQYM,3VADEH0UHCI5WYYIBGINV1X6N36SPQ,190,190,Approved
0,A29C1XYH77RQYM,3QD8LUVX4XJ9Z52YLBFWW39HBI7X5H,158,158,Approved


In [ ]:
turk.list_results("")

In [79]:
turk.approve_correct_hits()

In [7]:
# reset the hits
turk.approve_all_hits()
turk.stop_all_hits()
turk.delete_all_hits()

Stop hit 3T2EL38U0M5YSRLW533ZUE487EVQXO
Stop hit 3FVBZG9CLJZDI8GIJ1RYCMPW6KF0H8
Stop hit 3FK4G712NXL29QEA3XMHQ62HANESS7
Stop hit 36AZSFEYZ4LJKWECM65RB4CO53MVBE
Deleting hit 3T2EL38U0M5YSRLW533ZUE487EVQXO
Deleting hit 3FVBZG9CLJZDI8GIJ1RYCMPW6KF0H8
Deleting hit 3FK4G712NXL29QEA3XMHQ62HANESS7
Deleting hit 36AZSFEYZ4LJKWECM65RB4CO53MVBE


## Qualifications

In [ ]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=0
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN

In [ ]:
turker.client.update_qualification_type(
    QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN',
    QualificationTypeStatus='Active',
    Test='string',
    AnswerKey='string',
    TestDurationInSeconds=123,
    RetryDelayInSeconds=123,
    AutoGranted=False,
    AutoGrantedValue=123
)